# Generating percentiles

In this notebook we are going to generate the percentiles of a project. We are going to make use of the functionality explained in the notebook *Percentile_groups*.

Given a project, we can divide it into subgroups depending on the characteristics of the project. Once divided we will generate as many percentile groups as we have subgroups. 

The basic structure of subgroups is as follows:

```python
[
    {
        'dataframe': <pandas dataframe object>, # df of the subgroup
        'specie': 'homo sapiens',
        'organ': 'brain',
        'disease': 'parkinson'
    },
    {
        'dataframe': <pandas dataframe object>, # df of the subgroup
        'specie': 'homo sapiens',
        'organ': 'brain',
        'disease': 'brain cancer'
    }
]
```

In order to create the percentiles, we also need the expression matrix and the cell names of the matrix.

In [16]:
from Percentile_groups import read_files, process_metadata, get_groups_from_project_multiple

We are using the same project used in the *Percentile_groups* notebook. This project can be found in https://www.ebi.ac.uk/gxa/sc/experiments/E-MTAB-7678/experiment-design.

In [2]:
project_ID = 'E-MTAB-7678'

Before getting the subgroups, we can examine the files of this project.

In [3]:
metadata, matrix, gene_names, cell_names = read_files(project_ID)

In [5]:
print(f"In metadata we have information of {len(metadata)} cells.")
print(f"Our matrix has dimensions of {matrix.A.shape}.")
print(f"The matrix has {len(gene_names)} genes and {len(cell_names)} cells.")

In metadata we have information of 7052 cells.
Our matrix has dimensions of (7052, 19975).
The matrix has 19975 genes and 7052 cells.


In [11]:
metadata = process_metadata(metadata, cell_names)

In [13]:
cell_names['Assay'].equals(metadata['Assay'])

True

As we can see, the cells in the metadata are the same (and in the same order) as the cells in the matrix.

## Obtaining the subgroups

In [18]:
from Percentile_groups import get_groups_from_project_multiple, print_subgroups

In [14]:
common_characteristics = [
    'organism',
    'organism part',
    'sampling site',
    'biopsy site',
    'metastatic site',
    'developmental stage',
    'cell line',
    'disease'
]

characterictics_groups = [
    common_characteristics + ['cell type'],
    common_characteristics + ['inferred cell type - ontology labels'],
    common_characteristics + ['inferred cell type - authors labels'],
]

In [17]:
row, subgroups = get_groups_from_project_multiple(project_ID, characterictics_groups)

We can use the `print_subgroups` function to see how our subgroups looks like.

In [19]:
print_subgroups(subgroups)

Subgroup 0:
	Number of cells: 2718
	organism: Mus musculus
	organism part: lung
	developmental stage: adult
	cell type: classical monocyte
Subgroup 1:
	Number of cells: 4334
	organism: Mus musculus
	organism part: lung
	developmental stage: adult
	cell type: lung macrophage


## Calculating percentiles

Now we have the subgroups, we want to obtain the percentile of each one. To archieve that, we are first explaining what we are going to do step by step.

As we said before, the cells in the subgroups (part of the metadata) are the same and in the same order as the cells in the matrix. Knowing that, we can index the matrix to obtain the matrix with the cells of the subgroup.

In [21]:
sub = subgroups[0]
sub['dataframe']

,Assay,organism,strain,age,developmental stage,sex,genotype,organism part,cell type,immunophenotype
4334,SAMEA5367305-AAACCTGAGACTGGGT,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C hi CD64-
4335,SAMEA5367305-AAACCTGAGCTGAACG,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C hi CD64-
4336,SAMEA5367305-AAACCTGAGTAGGCCA,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C hi CD64-
4337,SAMEA5367305-AAACCTGTCCAAGCCG,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C hi CD64-
4338,SAMEA5367305-AAACGGGCAGCCAATT,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C hi CD64-
...,...,...,...,...,...,...,...,...,...,...
7047,SAMEA5367306-TTTGCGCTCCTAGGGC,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C lo CD64-
7048,SAMEA5367306-TTTGCGCTCCTCAATT,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C lo CD64-
7049,SAMEA5367306-TTTGGTTTCGGATGGA,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C lo CD64-
7050,SAMEA5367306-TTTGTCACAATGAATG,Mus musculus,C57BL/6J,10 week,adult,female,wild type genotype,lung,classical monocyte,CD45+ F4/80+ CD11c- Ly6C lo CD64-


In [26]:
sub_cells_index = sub['dataframe']['Assay'].index
print(len(sub_cells_index))

2718


In [29]:
sub_matrix = matrix.A[sub_cells_index]
print(f"The matrix of the subgroup has shape {sub_matrix.shape}")

The matrix of the subgroup has shape (2718, 19975)


In [31]:
import numpy as np

In [42]:
sub_matrix_mean = np.mean(sub_matrix, axis=0)
print(f"We have an array of mean expresion for {len(sub_matrix_mean)} genes. An example of the first 5 items:")
print(sub_matrix_mean[:5])

We have an array of mean expresion for 19975 genes. An example of the first 5 items:
[ 66.43592566   1.88408312   0.42475511   7.80919696 423.00130105]


However, when generating the percentiles, we do not want to take into account genes that have a mean expression of 0, as we consider that these genes are not expressed in these cells.

In [60]:
print(f"We have {np.sum(sub_matrix_mean == 0)} genes that are not expressed in our array.")

We have 314 genes that are not expressed in our array.


In [49]:
sub_matrix_mean_without_zeros = sub_matrix_mean[sub_matrix_mean != 0]
print(f"Removing zeros from the mean expression array, we have {len(sub_matrix_mean_without_zeros)} genes expressed.")

Removing zeros from the mean expression array, we have 19661 genes expressed.


### Explaining percentiles

In statistics, a percentile (or a centile) is a score below which a given percentage of scores in its frequency distribution falls (exclusive definition) or a score at or below which a given percentage falls (inclusive definition).

We are using the `scipy.stats` module function `percentileofscore`. In this function there are multiple types of percentiles. We will see an example of them.

More of this function in https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html

In [43]:
from scipy.stats import percentileofscore

In [52]:
l = [1, 2, 3, 3, 4]

print([percentileofscore(l, x, 'rank') for x in l])
print([percentileofscore(l, x, 'weak') for x in l])
print([percentileofscore(l, x, 'strict') for x in l])
print([percentileofscore(l, x, 'mean') for x in l])

[20.0, 40.0, 70.0, 70.0, 100.0]
[20.0, 40.0, 80.0, 80.0, 100.0]
[0.0, 20.0, 40.0, 40.0, 80.0]
[10.0, 30.0, 60.0, 60.0, 90.0]


We are interested in the *rank*, *weak* or *mean* percentile type, since we dont want any expressed gene to have a 0% percentile. We will select, for example, the *weak* type.

### Generate percentiles

As we have said, we will not take into account genes that are not expressed when calculating percentiles. However, these genes will have a percentile of 0% to indicate that they are not expressed.

In [56]:
sub_matrix_percentiles = [percentileofscore(sub_matrix_mean_without_zeros, x, 'weak') for x in sub_matrix_mean]
print(f"We have {len(sub_matrix_percentiles)} percentiles for the subgroup. We can see the first 5 percentiles:")
print(sub_matrix_percentiles[:5])

We have 19975 percentiles for the subgroup. We can see the first 5 percentiles:
[87.6354203753624, 28.838817964498247, 14.505874574029805, 48.28340369258939, 98.2554295305427]


## Saving percentiles

The best way to save the percentiles generated, since we want to insert them into the postgres database, is to build a dataframe and save is as a csv. There are two ways of build the dataframe that we are going to see. In both of them the gene_names are needed.

In [82]:
gene_names

,Gen_Name
0,ENSMUSG00000000001\tENSMUSG00000000001
1,ENSMUSG00000000028\tENSMUSG00000000028
2,ENSMUSG00000000037\tENSMUSG00000000037
3,ENSMUSG00000000056\tENSMUSG00000000056
4,ENSMUSG00000000058\tENSMUSG00000000058
...,...
19970,ENSMUSG00000118618\tENSMUSG00000118618
19971,ENSMUSG00000118623\tENSMUSG00000118623
19972,ENSMUSG00000118626\tENSMUSG00000118626
19973,ENSMUSG00000118633\tENSMUSG00000118633


And as we can see, the gene names are 'duplicated'. Fixing it is pretty simple.

In [85]:
gene_names['Gen_Name'] = gene_names['Gen_Name'].apply(lambda x: x.split('\t')[1])
gene_names

,Gen_Name
0,ENSMUSG00000000001
1,ENSMUSG00000000028
2,ENSMUSG00000000037
3,ENSMUSG00000000056
4,ENSMUSG00000000058
...,...
19970,ENSMUSG00000118618
19971,ENSMUSG00000118623
19972,ENSMUSG00000118626
19973,ENSMUSG00000118633


### Dense CSV

Here we create a CSV with a row per gene, and columns for the percentile, gene name and subgroup information.

In [57]:
import pandas as pd

In [139]:
n_genes = len(gene_names)

sub_copy = sub.copy()
del sub_copy['dataframe']

sub_matrix_results = pd.DataFrame(data = {
    'project_id': [project_ID] * n_genes,
    'metadata': [str(sub_copy)] * n_genes,
    'gene_name': gene_names['Gen_Name'],
    'percentile': sub_matrix_percentiles,
    'number_genes': [len(sub_matrix_mean_without_zeros)] * n_genes,
    'number_cells': [len(sub['dataframe'])] * n_genes
})

sub_matrix_results

,project_id,metadata,gene_name,percentile,number_genes,number_cells
0,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000000001,87.635420,19661,2718
1,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000000028,28.838818,19661,2718
2,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000000037,14.505875,19661,2718
3,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000000056,48.283404,19661,2718
4,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000000058,98.255430,19661,2718
...,...,...,...,...,...,...
19970,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000118618,1.571639,19661,2718
19971,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000118623,1.942933,19661,2718
19972,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000118626,48.263059,19661,2718
19973,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",ENSMUSG00000118633,59.183154,19661,2718


In [134]:
sub_matrix_results.to_csv(project_ID + '.v0.csv', index=False)

### Light CSV

Here, we create two dataframes, one with the percentiles and their genes and other with the subgroup information.

In [135]:
sub_matrix_results = pd.DataFrame(data = {
    'gene_name': gene_names['Gen_Name'],
    'percentile': sub_matrix_percentiles
})

sub_matrix_results

,gene_name,percentile
0,ENSMUSG00000000001,87.635420
1,ENSMUSG00000000028,28.838818
2,ENSMUSG00000000037,14.505875
3,ENSMUSG00000000056,48.283404
4,ENSMUSG00000000058,98.255430
...,...,...
19970,ENSMUSG00000118618,1.571639
19971,ENSMUSG00000118623,1.942933
19972,ENSMUSG00000118626,48.263059
19973,ENSMUSG00000118633,59.183154


In [140]:
sub_copy = sub.copy()
del sub_copy['dataframe']

sub_matrix_results_info = pd.DataFrame([{
    'project_id': project_ID,
    'metadata': str(sub_copy),
    'number_genes': len(sub_matrix_mean_without_zeros),
    'number_cells': len(sub['dataframe'])
}])

sub_matrix_results_info

,project_id,metadata,number_genes,number_cells
0,E-MTAB-7678,"{'organism': 'Mus musculus', 'organism part': ...",19661,2718


In [137]:
sub_matrix_results.to_csv(project_ID + '.v2.csv', index=False)
sub_matrix_results_info.to_csv(project_ID + '_info.v2.csv', index=False)

### Comparing CSVs

With the dense CSV we have all the information in one dataframe, and it make easier to insert the data into the database. We can even append this dense dataframe for the other subgroups of the project or even to other projects dataframes. However, with this approach we have a lot of information repeated unnecessarily, leading to an increase in the space it occupies.

On the other hand, we have the ligh CSV approach, here we will have more documents (2 per subgroup) but saving a lot of space. Also, the process of insertion into the database can be a bit more tedious as we have to navigate over more files.

In [142]:
%%bash
du -h E-MTAB-7678*

3,5M	E-MTAB-7678.v0.csv
724K	E-MTAB-7678.v2.csv
4,0K	E-MTAB-7678_info.v2.csv
